In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

house = pd.read_csv("train.csv")
houseTest = pd.read_csv("test.csv")

Id = houseTest["id"]
houseTest = houseTest.drop(["id"],axis=1)


# house
print("-"*50+ "train简单查看统计学信息" +"-"*50)
print(house.info())
print("-"*50+ "test简单查看统计学信息" +"-"*50)
print(houseTest.info())

print("-"*50 + "train查看缺失值" + "-"*50)
print(house.isnull().sum().sort_values(ascending=False).head(10))
print("-"*50 + "test查看缺失值" + "-"*50)
print(houseTest.isnull().sum().sort_values(ascending=False).head(10))



--------------------------------------------------train简单查看统计学信息--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196539 entries, 0 to 196538
Data columns (total 19 columns):
时间          196539 non-null int64
小区名         196539 non-null int64
小区房屋出租数量    195538 non-null float64
楼层          196539 non-null int64
总楼层         196539 non-null float64
房屋面积        196539 non-null float64
房屋朝向        196539 non-null object
居住状态        20138 non-null float64
卧室数量        196539 non-null int64
厅的数量        196539 non-null int64
卫的数量        196539 non-null int64
出租方式        24230 non-null float64
区           196508 non-null float64
位置          196508 non-null float64
地铁线路        91778 non-null float64
地铁站点        91778 non-null float64
距离          91778 non-null float64
装修情况        18492 non-null float64
月租金         196539 non-null float64
dtypes: float64(12), int64(6), object(1)
memory usage: 28.5+ MB
None
----------------------------------------

In [2]:
house["位置"][house["位置"].isnull()] = house["位置"].dropna().mode().values
house["区"][house["区"].isnull()] = house["区"].dropna().mode().values
house["小区房屋出租数量"][house["小区房屋出租数量"].isnull()] = house["小区房屋出租数量"].dropna().mode().values
houseTest["位置"][houseTest["位置"].isnull()] = houseTest["位置"].dropna().mode().values
houseTest["区"][houseTest["区"].isnull()] = houseTest["区"].dropna().mode().values
houseTest["小区房屋出租数量"][houseTest["小区房屋出租数量"].isnull()] = houseTest["小区房屋出租数量"].dropna().mode().values

print("-"*50 + "train查看缺失值" + "-"*50)
print(house.isnull().sum().sort_values(ascending=False).head(10))
print("-"*50 + "test查看缺失值" + "-"*50)
print(houseTest.isnull().sum().sort_values(ascending=False).head(10))



--------------------------------------------------train查看缺失值--------------------------------------------------
装修情况        178047
居住状态        176401
出租方式        172309
距离          104761
地铁站点        104761
地铁线路        104761
月租金              0
房屋面积             0
小区名              0
小区房屋出租数量         0
dtype: int64
--------------------------------------------------test查看缺失值--------------------------------------------------
装修情况        52072
居住状态        51796
出租方式        51308
地铁站点        29785
地铁线路        29785
距离          29785
房屋面积            0
小区名             0
小区房屋出租数量        0
楼层              0
dtype: int64


In [3]:
# 装修情况肯定对租金价格有较大影响，我们现在有两万的装修数据，可以尝试下预测
# 居住状态不太好预测，这个属于人为因素，但是应该有人居住的房子一定租金较低
# 出租方式，妥妥的有影响，中介收费不一，朋友介绍肯定便宜
# 距离应该是地铁站点到房子的距离，交通的影响
# 地铁站这个不太好预测，因为住房附近可能压根没地铁
# 小区出租数量，位置，区 可以直接干掉


In [4]:
# 特征工程

# 房屋朝向
print("-"*50+"house房屋朝向"+"-"*50)
house_orientation_unique = house["房屋朝向"].unique()
print(house["房屋朝向"].unique())
# print(house["房屋朝向"].value_counts())
print("-"*50+"houseTest房屋朝向"+"-"*50)
houseTest_orientation_unique = houseTest["房屋朝向"].unique()
print(houseTest["房屋朝向"].unique())
# print(houseTest["房屋朝向"].value_counts())
print()
print("house 有 houseTest没有:")
print ([i for i in house_orientation_unique if i not in houseTest_orientation_unique])
print("house 没有 houseTest有:")
print ([i for i in houseTest_orientation_unique if i not in house_orientation_unique])
print("house 有 houseTest有:")
print ([i for i in houseTest_orientation_unique if i in house_orientation_unique])

house_orientation = pd.DataFrame({'东':[0]*196539,'南':[0]*196539,
                                 '西':[0]*196539,'北':[0]*196539,
                                 '东南':[0]*196539,'东北':[0]*196539,
                                 '西南':[0]*196539,'西北':[0]*196539,})
houseTest_orientation = pd.DataFrame({'东':[0]*196539,'南':[0]*196539,
                                 '西':[0]*196539,'北':[0]*196539,
                                 '东南':[0]*196539,'东北':[0]*196539,
                                 '西南':[0]*196539,'西北':[0]*196539,})
house = house.join(house_orientation)
# house

# 将房屋朝向整成八个特征
def orientation_dong(x):
    lst = []
    for i in x.split(" "):
        lst.append(i)
    dong = "东"
    if dong in lst:
        return 1
    else:
        return 0

def orientation_nan(x):
    lst = []
    for i in x.split(" "):
        lst.append(i)
    nan = "南"
    if nan in lst:
        return 1
    else:
        return 0

    
def orientation_xi(x):
    lst = []
    for i in x.split(" "):
        lst.append(i)
    xi = "西"
    if xi in lst:
        return 1
    else:
        return 0


def orientation_bei(x):
    lst = []
    for i in x.split(" "):
        lst.append(i)
    bei = "北"
    if bei in lst:
        return 1
    else:
        return 0

    
def orientation_dongNan(x):
    lst = []
    for i in x.split(" "):
        lst.append(i)
    dongNan = "东南"
    if dongNan in lst:
        return 1
    else:
        return 0

    
def orientation_dongBei(x):
    lst = []
    for i in x.split(" "):
        lst.append(i)
    dongBei = "东北"
    if dongBei in lst:
        return 1
    else:
        return 0

    
def orientation_xiNan(x):
    lst = []
    for i in x.split(" "):
        lst.append(i)
    xiNan = "西南"
    if xiNan in lst:
        return 1
    else:
        return 0

    
def orientation_xiBei(x):
    lst = []
    for i in x.split(" "):
        lst.append(i)
    xiBei = "西北"
    if xiBei in lst:
        return 1
    else:
        return 0

    
house['东'] = house['房屋朝向'].apply(lambda x : orientation_dong(x) )
house['南'] = house['房屋朝向'].apply(lambda x : orientation_nan(x) )
house['西'] = house['房屋朝向'].apply(lambda x : orientation_xi(x) )
house['北'] = house['房屋朝向'].apply(lambda x : orientation_bei(x) )
house['东南'] = house['房屋朝向'].apply(lambda x : orientation_dongNan(x) )
house['东北'] = house['房屋朝向'].apply(lambda x : orientation_dongBei(x) )
house['西南'] = house['房屋朝向'].apply(lambda x : orientation_xiNan(x) )
house['西北'] = house['房屋朝向'].apply(lambda x : orientation_xiBei(x) )

houseTest['东'] = houseTest['房屋朝向'].apply(lambda x : orientation_dong(x) )
houseTest['南'] = houseTest['房屋朝向'].apply(lambda x : orientation_nan(x) )
houseTest['西'] = houseTest['房屋朝向'].apply(lambda x : orientation_xi(x) )
houseTest['北'] = houseTest['房屋朝向'].apply(lambda x : orientation_bei(x) )
houseTest['东南'] = houseTest['房屋朝向'].apply(lambda x : orientation_dongNan(x) )
houseTest['东北'] = houseTest['房屋朝向'].apply(lambda x : orientation_dongBei(x) )
houseTest['西南'] = houseTest['房屋朝向'].apply(lambda x : orientation_xiNan(x) )
houseTest['西北'] = houseTest['房屋朝向'].apply(lambda x : orientation_xiBei(x) )

house = house.drop("房屋朝向",axis=1)
houseTest = houseTest.drop("房屋朝向",axis=1)
# house


--------------------------------------------------house房屋朝向--------------------------------------------------
['东南' '东' '南' '东北' '西南' '南 北' '西北' '西' '北' '东南 西南 西' '东 南' '东 东南' '南 西南'
 '东南 西北' '东 西南' '东南 南 西南' '东 南 西' '西南 西北' '东 东北' '东南 南' '东 西' '南 东' '西南 西'
 '东南 西南' '南 西' '西南 北' '西 北' '北 东北' '西 西北' '东南 东北' '西北 东北' '西北 北' '西南 东北'
 '东 东南 南' '东南 西' '南 西北' '东 北' '南 东北' '东 南 北' '东 西 北' '东 东南 北' '西南 西 东北'
 '东南 南 北' '东 南 西 北' '东 西北' '东南 北' '东南 西南 西北' '东南 南 西南 西' '南 西 北' '南 西南 西'
 '东 西 东北' '东 东南 西南' '南 西南 北' '东 南 西北 北' '东 西北 北' '北 南' '西 西北 北'
 '东 东南 南 西南 西' '北 西' '南 北 东北' '东南 西 北' '东 西南 北' '东 南 西南' '南 西南 西 西北']
--------------------------------------------------houseTest房屋朝向--------------------------------------------------
['东' '东南' '南' '西北' '西南' '西' '东北' '北' '南 北' '东 南' '东 北' '南 西' '东 西' '东 东南'
 '东 西南 北' '南 西南' '东 西北' '西北 北 东北' '东南 南' '东南 南 北' '西南 北' '东南 东北' '东南 西南'
 '东 西南' '东南 西北' '南 西南 北' '南 东' '西南 西北' '西北 北' '东 南 北' '南 西南 西 西北' '南 西北'
 '东 东北' '西 北' '东 南 西 北' '西南 西' '西北 东北' '东南 北' '东南 南 西南'

In [5]:
print("-"*50 + "train查看缺失值" + "-"*50)
print(house.isnull().sum().sort_values(ascending=False).head(10))
print("-"*50 + "test查看缺失值" + "-"*50)
print(houseTest.isnull().sum().sort_values(ascending=False).head(10))


--------------------------------------------------train查看缺失值--------------------------------------------------
装修情况        178047
居住状态        176401
出租方式        172309
距离          104761
地铁站点        104761
地铁线路        104761
西北               0
小区名              0
小区房屋出租数量         0
楼层               0
dtype: int64
--------------------------------------------------test查看缺失值--------------------------------------------------
装修情况        52072
居住状态        51796
出租方式        51308
距离          29785
地铁站点        29785
地铁线路        29785
西北              0
卫的数量            0
小区名             0
小区房屋出租数量        0
dtype: int64


In [6]:
# house['装修情况'] = house['装修情况'].fillna(house['装修情况'].mean())
# house['居住状态'] = house['居住状态'].fillna(house['居住状态'].mean())
# house['出租方式'] = house['出租方式'].fillna(house['出租方式'].mean())
# house['距离'] = house['距离'].fillna(0)
# house['地铁站点'] = house['地铁站点'].fillna(0)
# house['地铁线路'] = house['地铁线路'].fillna(0)

# houseTest['装修情况'] = houseTest['装修情况'].fillna(houseTest['装修情况'].mean())
# houseTest['居住状态'] = houseTest['居住状态'].fillna(houseTest['居住状态'].mean())
# houseTest['出租方式'] = houseTest['出租方式'].fillna(houseTest['出租方式'].mean())
# houseTest['距离'] = houseTest['距离'].fillna(0)
# houseTest['地铁站点'] = houseTest['地铁站点'].fillna(0)
# houseTest['地铁线路'] = houseTest['地铁线路'].fillna(0)


In [7]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostRegressor
from sklearn import model_selection

x = house.drop(["月租金"],axis=1)
y = house.月租金

x_train,x_test, y_train,y_test = train_test_split(x,y,test_size=0.2, random_state=10)

c:\users\dell\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [ ]:
from catboost import CatBoostRegressor

clf_est =CatBoostRegressor(random_seed=1314)
# iterations=500,loss_function='RMSE',custom_metric='RMSE',eval_metric='RMSE',
#                        leaf_estimation_method='Gradient',depth=10,learning_rate=0.20
clf_param_grid = {'learning_rate':[0.15,0.2],
                  'loss_function': ['RMSE'], # 'Logloss' 
                 'custom_metric': ['RMSE'], 
                  'eval_metric': ['RMSE'], 
                  'leaf_estimation_method':['Newton'], 
                  'depth':[10]}
clf_grid = model_selection.GridSearchCV(clf_est, clf_param_grid, n_jobs=3, cv=3, verbose=1)
print("fitting")
clf_grid.fit(x_train, y_train)

print('BestParams: ' + str(clf_grid.best_params_))

In [16]:
print('Training:')
clf = CatBoostRegressor(
                    learning_rate = clf_grid.best_estimator_.learning_rate_,
                   loss_function = 'RMSE',
                   custom_metric = 'RMSE',
                   eval_metric = 'RMSE',
                   leaf_estimation_method = 'Newton',
                   random_seed=1314)
clf.fit(x_train, y_train)
print('Training Finished')

Training:
0:	learn: 8.9742040	total: 40.2ms	remaining: 40.2s
1:	learn: 7.9856664	total: 78.4ms	remaining: 39.1s
2:	learn: 7.1780427	total: 117ms	remaining: 38.8s
3:	learn: 6.5219200	total: 152ms	remaining: 37.8s
4:	learn: 5.9839008	total: 189ms	remaining: 37.6s
5:	learn: 5.5517924	total: 226ms	remaining: 37.5s
6:	learn: 5.2091665	total: 270ms	remaining: 38.3s
7:	learn: 4.9343948	total: 306ms	remaining: 38s
8:	learn: 4.7146117	total: 340ms	remaining: 37.5s
9:	learn: 4.5437751	total: 382ms	remaining: 37.8s
10:	learn: 4.4077428	total: 440ms	remaining: 39.6s
11:	learn: 4.2904481	total: 515ms	remaining: 42.4s
12:	learn: 4.1972638	total: 567ms	remaining: 43s
13:	learn: 4.1094837	total: 606ms	remaining: 42.7s
14:	learn: 4.0375863	total: 647ms	remaining: 42.5s
15:	learn: 3.9833101	total: 688ms	remaining: 42.3s
16:	learn: 3.9222042	total: 726ms	remaining: 42s
17:	learn: 3.8840677	total: 762ms	remaining: 41.6s
18:	learn: 3.8518102	total: 794ms	remaining: 41s
19:	learn: 3.8166193	total: 829ms	rem

In [17]:
predictions = clf.predict(houseTest.astype(float))
Submission = pd.DataFrame({'id': Id, 
                           'price': predictions})
Submission.to_csv('Submission.csv',index=False,sep=',')
print("I LOVE YOUU")


print("评分：")
from sklearn.model_selection import cross_val_score

scores = cross_val_score(clf,x_test,y_test)
print("Boost Score: ",scores.mean())
from sklearn import metrics

y_pred_xgb = clf.predict(x_test)
# 均方根误差
print("均方根误差: ",metrics.mean_squared_error(y_test,y_pred_xgb))

I LOVE YOUU
评分：
0:	learn: 9.0257921	total: 31.9ms	remaining: 31.9s
1:	learn: 8.0731792	total: 62.3ms	remaining: 31.1s
2:	learn: 7.3168657	total: 96.2ms	remaining: 32s
3:	learn: 6.6977187	total: 133ms	remaining: 33.2s
4:	learn: 6.1918990	total: 163ms	remaining: 32.5s
5:	learn: 5.7672714	total: 207ms	remaining: 34.3s
6:	learn: 5.4034234	total: 251ms	remaining: 35.6s
7:	learn: 5.1195786	total: 286ms	remaining: 35.4s
8:	learn: 4.9189468	total: 326ms	remaining: 35.9s
9:	learn: 4.7439550	total: 362ms	remaining: 35.8s
10:	learn: 4.6031510	total: 402ms	remaining: 36.2s
11:	learn: 4.4847744	total: 443ms	remaining: 36.5s
12:	learn: 4.3800531	total: 478ms	remaining: 36.3s
13:	learn: 4.3075380	total: 516ms	remaining: 36.4s
14:	learn: 4.2398123	total: 551ms	remaining: 36.2s
15:	learn: 4.1668195	total: 585ms	remaining: 36s
16:	learn: 4.1144930	total: 617ms	remaining: 35.6s
17:	learn: 4.0665242	total: 648ms	remaining: 35.4s
18:	learn: 4.0334286	total: 681ms	remaining: 35.2s
19:	learn: 3.9982424	total

In [ ]:


# clf = joblib.load('clf2.09.pkl')    # 下载模型
# print(clf)

# print("评分：")
# from sklearn.model_selection import cross_val_score

# scores = cross_val_score(clf,x_test,y_test)
# print("Boost: ",scores.mean())

In [ ]:
from sklearn.externals import joblib

# joblib.dump(clf, 'clf2.09.pkl')       # 保存模型，需先建立saved_model文件夹

# clf = joblib.load('clf2.09.pkl')    # 下载模型
# print(clf)